In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split , cross_val_score , RandomizedSearchCV
from sklearn.preprocessing import StandardScaler ,MinMaxScaler ,LabelEncoder
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/retencion-en-telefonia-movil-2024-10/testelco.xlsx
/kaggle/input/retencion-en-telefonia-movil-2024-10/traintelco.xlsx
/kaggle/input/retencion-en-telefonia-movil-2024-10/samplesubtelco.csv


In [2]:
Ruta = '/kaggle/input/retencion-en-telefonia-movil-2024-10'
train_data = pd.read_excel('/kaggle/input/retencion-en-telefonia-movil-2024-10/traintelco.xlsx', sheet_name='train')
test_data = pd.read_excel('/kaggle/input/retencion-en-telefonia-movil-2024-10/testelco.xlsx', sheet_name='test')
answers_data = pd.read_csv('/kaggle/input/retencion-en-telefonia-movil-2024-10/samplesubtelco.csv')

In [3]:
train_data.columns

Index(['id', 'Fecha de nacimiento', 'tipo cliente', 'Factura online',
       'Antigüedad Equipo', 'Plan de datos', 'facturación', 'mora',
       'Fecha inicio contrato', 'minutos', 'resultado'],
      dtype='object')

In [4]:
test_data

,id,Fecha de nacimiento,tipo cliente,Factura online,Antigüedad Equipo,Plan de datos,facturación,mora,Fecha inicio contrato,minutos
0,13,1996-10-28 20:37:48.858,2,0,18,1,319690,0,2016-08-01 18:35:57.816,1202
1,14,1985-11-08 12:29:16.237,2,1,6,1,333812,0,2016-04-01 19:20:55.326,1403
2,20,1987-08-15 05:31:09.530,2,0,35,1,224201,43,2017-10-12 05:47:50.487,659
3,30,1981-11-06 04:06:28.190,1,1,11,1,210614,33,2018-02-09 20:47:49.340,912
4,37,1986-08-03 13:43:36.300,1,0,34,1,267116,15,2017-02-01 17:42:17.169,1622
...,...,...,...,...,...,...,...,...,...,...
1995,18270,1955-02-20 12:17:36.924,1,0,36,1,281281,31,2017-11-09 02:43:33.494,548
1996,18278,1955-05-09 08:52:52.158,1,0,37,1,332863,0,2016-04-25 09:51:40.711,1378
1997,18292,1954-05-26 00:07:33.824,2,1,8,1,287664,44,2018-01-24 21:20:35.166,484
1998,18295,1959-05-21 20:24:36.221,1,0,42,1,422019,1,2017-02-13 17:42:49.031,1563


In [5]:
answers_data

,id,resultado
0,13,0
1,14,0
2,20,0
3,30,0
4,37,0
...,...,...
1995,18270,0
1996,18278,0
1997,18292,0
1998,18295,0


EDA

Limpieza de datos

In [6]:
#limpiar
hoy = pd.Timestamp.now()
def fecha_na(df):
    df['edad'] = hoy.year - df['Fecha de nacimiento'].dt.year
    
def fecha_co(df):
    df['contrato'] = (hoy - df['Fecha inicio contrato']).dt.days
    
def eliminar(df):
    df.drop(['Fecha de nacimiento','id','Fecha inicio contrato'],axis =1 , inplace = True)


fecha_na(train_data)
fecha_na(test_data)
fecha_co(train_data)
fecha_co(test_data)
eliminar(train_data)
eliminar(test_data)
train_data

,tipo cliente,Factura online,Antigüedad Equipo,Plan de datos,facturación,mora,minutos,resultado,edad,contrato
0,2,1,9,0,44588,34,644,0,34,2330
1,2,0,41,1,341480,3,663,0,44,2477
2,2,0,38,1,230662,20,1039,1,40,2506
3,2,1,11,1,205336,9,535,0,38,2614
4,1,1,6,0,429525,0,1869,0,38,3203
...,...,...,...,...,...,...,...,...,...,...
8238,2,1,7,1,196444,27,8894,1,63,2714
8239,1,1,13,0,257538,3,662,0,62,2587
8240,2,1,10,1,288037,41,1465,0,63,2319
8241,1,0,33,1,207627,3,494,0,77,2466


In [7]:
train_data

,tipo cliente,Factura online,Antigüedad Equipo,Plan de datos,facturación,mora,minutos,resultado,edad,contrato
0,2,1,9,0,44588,34,644,0,34,2330
1,2,0,41,1,341480,3,663,0,44,2477
2,2,0,38,1,230662,20,1039,1,40,2506
3,2,1,11,1,205336,9,535,0,38,2614
4,1,1,6,0,429525,0,1869,0,38,3203
...,...,...,...,...,...,...,...,...,...,...
8238,2,1,7,1,196444,27,8894,1,63,2714
8239,1,1,13,0,257538,3,662,0,62,2587
8240,2,1,10,1,288037,41,1465,0,63,2319
8241,1,0,33,1,207627,3,494,0,77,2466


In [8]:
print('Distribucion de clases antes:', Counter(train_data['resultado']))
rus = RandomUnderSampler(random_state=42)
x = train_data.drop(['resultado'] ,axis =1)
y = train_data['resultado']
X_res, y_res = rus.fit_resample(x,y)
print('Distribucion de clases antes:', Counter(y_res))

Distribucion de clases antes: Counter({0: 6882, 1: 1361})
Distribucion de clases antes: Counter({0: 1361, 1: 1361})


Modelo

In [9]:
x = X_res
y = y_res

In [10]:
state = 1
size = 0.20
x_train ,x_test , y_train , y_test = train_test_split(x,y,test_size=size , random_state =state,stratify=y)

In [11]:
xgb = XGBClassifier(
base_score=0.6, booster='gbtree',
    learning_rate=0.02, max_depth=10,
    min_child_weight=8, n_estimators=505,
    subsample=0.5, tree_method='exact',
    validate_parameters=0.5, verbosity=None)
print(xgb.fit(x_train , y_train))
xgb_pred = xgb.predict(x_test)

XGBClassifier(base_score=0.6, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.02, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=8, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=505, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)


In [12]:
print("Acc of XGBClassifier ", accuracy_score(y_test,xgb_pred))

Acc of XGBClassifier  0.7834862385321101


In [13]:
from sklearn.metrics import confusion_matrix
results = confusion_matrix(y_test, xgb_pred)
print(results)

[[212  60]
 [ 58 215]]


In [14]:
#XGB CLASSIFIER: 
test_pred = xgb.predict(test_data)

In [15]:
test_data

,tipo cliente,Factura online,Antigüedad Equipo,Plan de datos,facturación,mora,minutos,edad,contrato
0,2,0,18,1,319690,0,1202,28,2838
1,2,1,6,1,333812,0,1403,39,2960
2,2,0,35,1,224201,43,659,37,2401
3,1,1,11,1,210614,33,912,43,2281
4,1,0,34,1,267116,15,1622,38,2654
...,...,...,...,...,...,...,...,...,...
1995,1,0,36,1,281281,31,548,69,2373
1996,1,0,37,1,332863,0,1378,69,2936
1997,2,1,8,1,287664,44,484,70,2297
1998,1,0,42,1,422019,1,1563,65,2642


In [16]:
test_pred

array([0, 1, 0, ..., 1, 0, 1])

In [17]:
answers_data['resultado'] = test_pred

In [18]:
answers_data

,id,resultado
0,13,0
1,14,1
2,20,0
3,30,0
4,37,0
...,...,...
1995,18270,0
1996,18278,1
1997,18292,1
1998,18295,0


In [19]:
answers_data.to_csv("submission.csv" , index = False)